In [52]:
import os, csv
import pandas as pd
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# convert all txt files into a single csv file
corpus = './corpus.csv'
txt_files_folder = './train-mails'
with open(corpus, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['File Name', 'Content'])

    for txt_filename in os.listdir(txt_files_folder):
        if txt_filename.endswith('.txt'):
            with open(os.path.join(txt_files_folder, txt_filename), 'r', encoding='utf-8') as txt_file:
                content = txt_file.read()
                csv_writer.writerow([txt_filename, content])

In [ ]:
# Create a dictionary of words (dropping all non-words like punctuation characters, single characters)
# choosing the 2000 most frequent words from training set
csv_file = pd.read_csv('./corpus.csv', encoding='utf-8')
csv_file['Content'] = csv_file['Content'].str.replace(r'[^\w\s]+|(?<=\s)\S(?=\s)|\S(?<=\s)','',regex=True)
dictionary = {}
for i in range(len(csv_file['Content'])):
    #TODO: remove subject from dictionary
    for word in csv_file['Content'][i].split():
        if word not in dictionary:
            if len(word) > 1:
                dictionary[word] = 1
        elif word in dictionary:
            dictionary[word] += 1
    del dictionary['Subject']
    

top_2000_words = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)[:2000]
sortedDictionary = dict(top_2000_words)
for word in top_2000_words:
    print(word)

In [ ]:
print(csv_file['Content'][0])

In [ ]:
# Transform each mail into a word count vector basing on the dictionary of words
#word_count_matrix = CountVectorizer(vocabulary=sortedDictionary.keys()).transform(csv_file['Content'])

matrix = []

for mail_index in range(len(csv_file['Content'])):
    matrix.append([])
    for key in sortedDictionary.keys():
        word_counter = 0
        for word in csv_file['Content'][mail_index].split():
            if word == key:
                word_counter += 1
        matrix[-1].append(word_counter)

print('Matrix dimensions:', len(matrix), 'x', len(matrix[0]))

In [50]:
print(len(matrix))


702


In [ ]:
# Train a Bayes classifier (MultinomialNB from sklearn.naive_bayes)
# Check with the test set the classifier

X, y = make_classification(
    n_features=2000,
    n_classes=2,
    n_samples=702,
    n_clusters_per_class=1,
)

plt.scatter(X[:, 0], X[:, 1], c=y, marker="*")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=125
)


model = MultinomialNB()

model.fit(X_train, y_train)

# TODO: finish implementation